# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("output_data/weather_data.csv")
cities_df = cities_df.set_index('index')
cities_df.head()

,City,Lat,Lng,Temperature,Humidity,Cloudiness,Wind_Speed,Country,Date
index,,,,,,,,,
0,jonesboro,35.8423,-90.7043,78.03,41,1,19.57,US,1619477469
1,tambilil,5.9622,124.6697,81.30,78,93,2.55,PH,1619477480
2,alice springs,-23.7000,133.8833,53.60,50,0,3.44,AU,1619477481
3,ponta do sol,32.6667,-17.1000,56.21,78,78,1.01,PT,1619477212
4,punta arenas,-53.1500,-70.9167,44.29,93,90,5.75,CL,1619476936


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations and humidities in humidities
locations = cities_df[["Lat", "Lng"]]
humidities = cities_df.Humidity

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Create heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidities, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)



fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacation_df = cities_df.loc[(cities_df.Temperature >= 70) & 
                            (cities_df.Temperature <= 80) &
                            (cities_df.Wind_Speed <= 10) &
                            (cities_df.Cloudiness <= 0), :]
vacation_df.head(20)

,City,Lat,Lng,Temperature,Humidity,Cloudiness,Wind_Speed,Country,Date
index,,,,,,,,,
388,bundaberg,-24.8500,152.3500,71.38,72,0,9.22,AU,1619477603
477,baracoa,20.3467,-74.4958,78.78,75,0,7.49,CU,1619477636
486,umm lajj,25.0213,37.2685,77.52,73,0,6.17,SA,1619477639
500,karratha,-20.7377,116.8463,74.84,33,0,9.01,AU,1619477312


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create empty hotel dataframe
hotel_df = pd.DataFrame({"Hotel_Name" : []})
hotel_df.head()

,Hotel_Name


In [7]:
#  Set up url for Google Places API Calls
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location" : "",
    "radius" : 5000,
    "type": "lodging",
    "key": g_key,
}

In [8]:
# For loop to get city coords and make API calls
for index, row in vacation_df.iterrows():
    lat = row.Lat
    lng = row.Lng
    params['location'] = f"{lat}, {lng}"
    response = requests.get(base_url, params=params).json()
    hotel_df.loc[index, "Hotel_Name"] = response["results"][0]["name"]
    hotel_df.loc[index, "City"] = row.City.title()
    hotel_df.loc[index, "Country"] = row.Country
    hotel_df.loc[index, "Lat"] = response["results"][0]["geometry"]["location"]["lat"]
    hotel_df.loc[index, "Lng"] = response["results"][0]["geometry"]["location"]["lng"]
   


In [9]:
hotel_df.head()

,Hotel_Name,City,Country,Lat,Lng
388,Takalvan Motel,Bundaberg,AU,-24.872265,152.333805
477,Hostal Rio Miel,Baracoa,CU,20.348122,-74.495257
486,Moon light Furnished Units,Umm Lajj,SA,25.017307,37.271824
500,ibis Styles Karratha,Karratha,AU,-20.732225,116.846744


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add hotel info pop up and markers on top of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))